In [ ]:
#[Joe] 1. Acquire and process stock data
#[Alejandra]x 2. Acquire and process sentiment analysis data
#[Session] 3. Run baseline
#[Joe]x 4. Run sentiment analysis with LinearDiscreminateAnalysis
#[Edward] 5. Run sentiment analysis with alternate classifier
#[Edward] 6. Combine results
#[Session] 7. Put together PPT preso

In [1]:
import datetime as dt
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.decomposition import PCA

In [2]:
# Load the CSV file
reuters = Path('./Resources/reuters_headlines.csv')
df_reuters = pd.read_csv(reuters)

# Convert the "Time" column to datetime
df_reuters['Time'] = pd.to_datetime(df_reuters['Time'])

# Sort the DataFrame by the "Time" column in ascending order
df_reuters.sort_values(by='Time', inplace=True)

# Set the "Time" column as the index
df_reuters.set_index('Time', inplace=True)

# Drop the "Description" column in place
df_reuters.drop(columns=['Description'], inplace=True)

df_reuters.dropna(inplace=True)

# Remove the extra timestamp from the "Time" column
df_reuters.index = df_reuters.index.date


df_reuters_copy = df_reuters.copy()
df_reuters_copy

,Headlines
2018-03-20,UK will always consider ways to improve data l...
2018-03-20,Senate Democrat wants Facebook CEO Zuckerberg ...
2018-03-20,"Factbox: How United States, others regulate au..."
2018-03-20,Cambridge Analytica played key Trump campaign ...
2018-03-20,Start of AT&T-Time Warner trial delayed until ...
...,...
2020-07-17,Exclusive: Pact to aid poor cocoa farmers in p...
2020-07-18,Twitter says attackers downloaded data from up...
2020-07-18,Trail of missing Wirecard executive leads to B...
2020-07-18,Disney cuts ad spending on Facebook amid growi...


In [3]:
# Load the CSV file for CNBC headlines
cnbc = Path('./Resources/cnbc_headlines.csv')
df_cnbc = pd.read_csv(cnbc)

# Convert the "Time" column to datetime
df_cnbc['Time'] = pd.to_datetime(df_cnbc['Time'])

# Sort the DataFrame by the "Time" column in ascending order
df_cnbc.sort_values(by='Time', inplace=True)

# Set the "Time" column as the index
df_cnbc.set_index('Time', inplace=True)

# Drop the "Description" column in place
df_cnbc.drop(columns=['Description'], inplace=True)

# Drop rows with missing values (NaN)
df_cnbc.dropna(inplace=True)

# Remove the extra timestamp from the "Time" column
df_cnbc.index = df_cnbc.index.date

df_cnbc_copy = df_cnbc.copy()

df_cnbc_copy

,Headlines
2017-12-22,Cramer: Never buy a stock all at once — you'll...
2017-12-22,Cramer: I helped investors through the 2010 fl...
2017-12-22,Cramer says owning too many stocks and too lit...
2017-12-26,Markets lack Christmas cheer
2017-12-27,S&P tends to start new year bullish after this...
...,...
2020-07-16,Acorns CEO: Parents can turn $5 into five figu...
2020-07-17,IQ Capital CEO Keith Bliss says tech and healt...
2020-07-17,"Cramer's week ahead: Big week for earnings, ev..."
2020-07-17,Cramer's lightning round: I would own Teradyne


In [4]:

# Load the CSV file for The Guardian headlines
guardian = Path('./Resources/guardian_headlines.csv')
df_guardian = pd.read_csv(guardian)

# Convert the "Time" column to datetime with errors='coerce'
df_guardian['Time'] = pd.to_datetime(df_guardian['Time'], errors='coerce')

# Drop rows with NaT (invalid) timestamps
df_guardian.dropna(subset=['Time'], inplace=True)

# Sort the DataFrame by the "Time" column in ascending order
df_guardian.sort_values(by='Time', inplace=True)

# Set the "Time" column as the index
df_guardian.set_index('Time', inplace=True)

# Drop the "Description" column in place
# df_guardian.drop(columns=['Description'], inplace=True)


# Remove the extra timestamp from the "Time" column
df_guardian.index = df_guardian.index.date


df_guardian_copy = df_guardian.copy()
df_guardian_copy

,Headlines
2017-12-17,The Guardian view on Ryanair’s model: a union-...
2017-12-17,Peter Preston on press and broadcasting \n\n\...
2017-12-17,Why business could prosper under a Corbyn gove...
2017-12-17,Youngest staff to be given UK workplace pensio...
2017-12-17,Grogonomics \n\n\n This year has been about ...
...,...
2020-07-18,World Bank calls on creditors to cut poorest n...
2020-07-18,British Airways retires Boeing 747 fleet as Co...
2020-07-18,What will changes to England's lockdown rules ...
2020-07-18,St Mawes named UK’s top seaside resort in Whic...


In [5]:
sp500 = Path('./Resources/sp500.csv')

df_sp500 = pd.read_csv(sp500)

# Convert the "Time" column to datetime with errors='coerce'
df_sp500['Date'] = pd.to_datetime(df_sp500['Date'], errors='coerce')

# Drop rows with NaT (invalid) timestamps
df_sp500.dropna(subset=['Date'], inplace=True)

# Sort the DataFrame by the "Time" column in ascending order
df_sp500.sort_values(by='Date', inplace=True)

# Set the "Time" column as the index
df_sp500.set_index('Date', inplace=True)

df_sp500 = df_sp500.rename_axis('Time')

df_sp500 = df_sp500.drop(columns=['Open', 'High', 'Low', 'Adj Close'])

df_sp500_copy = df_sp500.copy()
df_sp500_copy = df_sp500_copy.rename(columns={'Close': 'SP500 Close'})
df_sp500_copy


,SP500 Close,Volume
Time,,
2017-12-01,264.459991,164390900
2017-12-04,264.140015,94040600
2017-12-05,263.190002,77994500
2017-12-06,263.239990,75898600
2017-12-07,264.070007,77218600
...,...,...
2020-07-13,314.839996,102997500
2020-07-14,318.920013,93657000
2020-07-15,321.850006,87196500


In [6]:
tsla = Path('./Resources/tsla.csv')


df_tsla = pd.read_csv(tsla)

# Convert the "Time" column to datetime with errors='coerce'
df_tsla['Date'] = pd.to_datetime(df_tsla['Date'], errors='coerce')

# Drop rows with NaT (invalid) timestamps
df_tsla.dropna(subset=['Date'], inplace=True)

# Sort the DataFrame by the "Time" column in ascending order
df_tsla.sort_values(by='Date', inplace=True)

# Set the "Time" column as the index
df_tsla.set_index('Date', inplace=True)
# Rename the "Close" column to "Tsla Close"

df_tsla = df_tsla.rename_axis('Time')

df_tsla = df_tsla.drop(columns=['Open', 'High', 'Low', 'Adj Close'])


df_tsla_copy = df_tsla.copy()
df_tsla_copy = df_tsla_copy.rename(columns={'Close': 'TSLA Close'})
df_tsla_copy


,TSLA Close,Volume
Time,,
2017-12-01,20.435333,64393500
2017-12-04,20.346666,87526500
2017-12-05,20.246668,69697500
2017-12-06,20.884001,107929500
2017-12-07,20.749332,71709000
...,...,...
2020-08-24,134.279999,300954000
2020-08-25,134.889328,159883500
2020-08-26,143.544662,213591000


In [7]:

# Merge DataFrames by the "Time" index
combined_stock = pd.concat((df_sp500_copy, df_tsla_copy), axis=1)

# Backfill NaN values 
combined_stock = combined_stock.fillna(method='ffill')

combined_stock_copy = combined_stock.copy()
combined_stock_copy.reset_index(inplace=True)  # Reset the index to make 'Time' a regular column
combined_stock_copy.drop_duplicates(subset=['Time'], keep='first', inplace=True)  # Drop duplicates based on 'Time'
combined_stock_copy = combined_stock_copy.set_index('Time')  # Set the index to 'Time' permanently

combined_stock_copy

,SP500 Close,Volume,TSLA Close,Volume
Time,,,,
2017-12-01,264.459991,164390900.0,20.435333,64393500
2017-12-04,264.140015,94040600.0,20.346666,87526500
2017-12-05,263.190002,77994500.0,20.246668,69697500
2017-12-06,263.239990,75898600.0,20.884001,107929500
2017-12-07,264.070007,77218600.0,20.749332,71709000
...,...,...,...,...
2020-08-24,321.720001,62774900.0,134.279999,300954000
2020-08-25,321.720001,62774900.0,134.889328,159883500
2020-08-26,321.720001,62774900.0,143.544662,213591000


In [8]:

data_frames = [df_reuters_copy, df_cnbc_copy, df_guardian_copy]

# Concatenate the DataFrames vertically
combined_df = pd.concat(data_frames, axis=0)

# Reset the index 
combined_df.reset_index(inplace=True)

# Rename the columns 
combined_df.rename(columns={'index': 'Time', 'Headlines': 'Headline'}, inplace=True)

# Group by the 'Time' column and aggregate the headlines into a single cell
combined_df = combined_df.groupby('Time')['Headline'].agg(' '.join).reset_index()

combined_df = combined_df.set_index('Time')

combined_headlines_copy = combined_df.copy()


combined_headlines_copy = combined_df.copy()
combined_headlines_copy

,Headline
Time,
2017-12-17,The Guardian view on Ryanair’s model: a union-...
2017-12-18,Brace yourself for the next Brexit faultline: ...
2017-12-19,"First pilots, now cabin crew – Ryanair to reco..."
2017-12-20,"Nearly 45,000 UK retailers in financial distre..."
2017-12-21,Last-minute Christmas shopping guide: there is...
...,...
2020-07-14,"Google hit with 600,000 euro Belgian privacy f..."
2020-07-15,Oil climbs 2% on U.S. stock draw but gains cap...
2020-07-16,Morgan Stanley posts record profit on trading ...


In [10]:

# Convert the 'Time' index to a datetime object in both DataFrames
combined_headlines_copy.index = pd.to_datetime(combined_headlines_copy.index)
combined_stock_copy.index = pd.to_datetime(combined_stock_copy.index)

# Concatenate the DataFrames along the columns axis (axis=1) using the 'Time' index
combined_result = pd.concat([combined_headlines_copy, combined_stock_copy], axis=1)

# Drop rows with NaN values in any column
combined_result.dropna(how='any', inplace=True)

combined_result.to_csv('combined_csv.csv')

combined_result 

,Headline,SP500 Close,Volume,TSLA Close,Volume
Time,,,,,
2017-12-18,Brace yourself for the next Brexit faultline: ...,268.200012,83653600.0,22.591333,82143000.0
2017-12-19,"First pilots, now cabin crew – Ryanair to reco...",267.170013,82382900.0,22.073334,102375000.0
2017-12-20,"Nearly 45,000 UK retailers in financial distre...",267.029999,76751500.0,21.931999,89307000.0
2017-12-21,Last-minute Christmas shopping guide: there is...,267.579987,67032300.0,22.110666,65778000.0
2017-12-22,Cramer: Never buy a stock all at once — you'll...,267.510010,78720900.0,21.680000,63237000.0
...,...,...,...,...,...
2020-07-13,"Luckin Coffee ousts chairman Lu, names Guo as ...",314.839996,102997500.0,99.804001,584781000.0
2020-07-14,"Google hit with 600,000 euro Belgian privacy f...",318.920013,93657000.0,101.120003,351271500.0
2020-07-15,Oil climbs 2% on U.S. stock draw but gains cap...,321.850006,87196500.0,103.067329,245517000.0
